# Aplicación de prueba

In [15]:
import pandas as pd
import joblib

from datetime import datetime
from flask import Flask, request


In [ ]:
# Definiciones globales y funciones de transformación

%run ../Lib/pandas_options.py
%run ../Lib/define_constantes.py
%run ../Lib/fn_transformaciones.py


In [ ]:
# Cargar archivo de tramos

archivo_tramos = f'{DIRECTORIO_DATOS}/DATOSTRAMOS_2023-08-06.csv'
df_tramos = pd.read_csv(archivo_tramos, low_memory=False)

print(df_tramos.info())

In [ ]:
# Cargar modelo

archivo_modelo = f'{DIRECTORIO_MODELOS}/modelo.joblib'
modelo = joblib.load(archivo_modelo)

print(modelo)

In [ ]:
# Cargar columnas

columnas_modelo = []
archivo_columnas = f'{DIRECTORIO_MODELOS}/COLUMNAS.txt'

with open(archivo_columnas, "r") as archivo:
    for linea in archivo:
        cadena = linea.strip()
        columnas_modelo.append(cadena)

print(columnas_modelo)


In [6]:
# Generar predicción

def generar_prediccion(ipid):

    df_predecir = df_tramos[df_tramos['IPID'] == ipid].copy()

    # Agregar columnas dummy para codificacion
    df_predecir['CALIFICACION'] = 1 # Irrelevante, no se usa pero es requerido para transformacion
    df_predecir['FECHA_CALIFICACION'] = datetime.now()
  
    X_predecir = tx_codificar(tx_aplicar_transformaciones(df_predecir))

    # Agregar columnas faltantes
    for col in columnas_modelo:
        if col not in X_predecir.columns:
            print(f'Agregando {col}')
            X_predecir[col] = False
    
    # Reordenar columnas
    X_predecir = X_predecir[columnas_modelo]
    
    probabilidades = modelo.predict_proba(X_predecir)
    probabilidad_deterioro = probabilidades[:, 1]
        
    return f'Probabilidad de estado DETERIORADO en tramo {ipid}: {probabilidad_deterioro[0]:.2%}'
    

In [7]:
# Generar mensaje de respuesta

def generar_respuesta(texto_consulta):
    try:
        ipid = int(texto_consulta)
    except ValueError:
        return 'IPID debe ser numérico'
        
    if ipid in df_tramos['IPID'].values:
        return generar_prediccion(ipid)
    else:
        return f"No encuentro tramo {ipid} en archivo de tramos"
    

In [ ]:
# Prototipo de servicio de consulta

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def predecir():
    if request.method == 'POST':
        texto_consulta = request.form['ipid']
        mensaje = generar_respuesta(texto_consulta)
        return f'''
        <p>{mensaje}</p>
        <form method="GET">
            <input type="submit" value="Consultar otro IPID">
        </form>
        '''
    else:
        return '''
        <h1>PREDICCIÓN DE ESTADO DE TRAMOS DE TUBERÍA</h1>
        <form method="POST">
            <label for="ipid">Ingresa el IPID del tramo:</label>
            <input type="text" id="ipid" name="ipid" required>
            <input type="submit" value="Enviar">
        </form>
        '''

if __name__ == '__main__':
    app.run()

